# Color Assimilation
the color-assimilated pictures are available at `./outputs` using different methods

In [1]:
import cv2 as cv
import numpy as np
import os

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
path = "./inputs"

images_paths = os.listdir(path)

rgb_images = []

for image_path in images_paths:
    I = cv.imread(os.path.join(path,image_path))
    rgb_images.append(I)

    print(f"{image_path} shape: {I.shape}")

AmirKabir.jpg shape: (2000, 1500, 3)
AUT-CE.jpg shape: (600, 1394, 3)


In [3]:
# Converting Colored Images to Grayscale
gray_images = []

# Coefficients used in cvtColor method for conveting BGR2GRAY
coefs = [0.114, 0.587, 0.299]

for i, image in enumerate(rgb_images):
    gray_images.append(cv.cvtColor(image, cv.COLOR_BGR2GRAY))
    
    print(f"{images_paths[i]} shape: {gray_images[i].shape}")

AmirKabir.jpg shape: (2000, 1500)
AUT-CE.jpg shape: (600, 1394)


### Adding Circles to Grayscale Image

In [ ]:
kernel_size = 20
radius = int(np.floor(0.4 * (kernel_size // 2)))
circle_density = 2 * radius + 2
opacity = 1

colorized_images = []
masks = []

for idx, image in enumerate(gray_images):
    h, w = image.shape
    print(f"Proccessing {images_paths[idx]}: shape = {image.shape}")

    I = np.concatenate(
        (
            image[..., np.newaxis],
            image[..., np.newaxis],
            image[..., np.newaxis],
        ),
        axis=2,
    )

    I_clone = I.copy()
    mask = np.zeros_like(I)

    row = radius + 2
    col = radius + 2

    diagonal = True

    while row < h:
        while col < w:
            # Calculate the center of the current circle
            center = (col, row)

            # Adding Circles to the mask
            cv.circle(mask, center, radius, (1, 1, 1), thickness=-1)

            # Updating the center position
            col += circle_density
        
        # Controling diagonal grid shape of circles
        if diagonal:
            col = (radius + 2) + circle_density // 2
            diagonal = False
        else:
            col = (radius + 2)
            diagonal = True
        row += circle_density
    
    # Color Assimation Phase   
    mask = (opacity * mask).astype(np.float32)
    inverse_mask = 1 - mask

    I = ((mask * rgb_images[idx]) + (I * inverse_mask)).astype(np.uint8)
    cv.imwrite(f"./outputs/{images_paths[idx].split(".")[0]}_colorized_circle.png", I)
    
    # Saving the Result
    colorized_images.append(I)
    masks.append(mask)
    
    # Showing the result
    final_image = np.concatenate([I_clone, I], axis=1)
    final_image_resized = cv.resize(final_image, (w, h//2))
    cv.imshow("pic", final_image_resized)
    cv.waitKey(0)
    cv.destroyAllWindows()
        

Proccessing AmirKabir.jpg: shape = (2000, 1500)
Proccessing AUT-CE.jpg: shape = (600, 1394)


### Adding Rhombus to Grayscale Image

In [ ]:
kernel_size = 40
d1, d2 = 12, 8 # Rhombus diagonal length
rhombus_density = abs(d1-d2) * 2 + 3
opacity = 1

for idx, image in enumerate(gray_images):
    h, w = image.shape
    print(f"Proccessing {images_paths[idx]}: shape = {image.shape}")

    I = np.concatenate(
        (
            image[..., np.newaxis],
            image[..., np.newaxis],
            image[..., np.newaxis],
        ),
        axis=2,
    )

    I_clone = I.copy()
    mask = np.zeros_like(I)

    row = kernel_size // 2
    col = kernel_size // 2

    diagonal = True

    while row < h:
        while col < w:
            # Calculating the center of the current rhombus
            center = (col, row)

            # Calculate the vertices based on the center and diagonals
            half_d1, half_d2 = d1 // 2, d2 // 2
            vertices = np.array([
                [center[0] - half_d2, center[1]],         # Left vertex
                [center[0], center[1] - half_d1],         # Top vertex
                [center[0] + half_d2, center[1]],         # Right vertex
                [center[0], center[1] + half_d1]          # Bottom vertex
            ], np.int32)

            # Reshape vertices for cv2.polylines
            vertices = vertices.reshape((-1, 1, 2))

            # Drawing the rhombus
            cv.fillPoly(mask, [vertices], color=(1, 1, 1))

            # Updating the center position
            col += rhombus_density
        
        # Controling diagonal grid shape of rhombus
        if diagonal:
            col = kernel_size // 2 + rhombus_density // 2
            diagonal = False
        else:
            col = kernel_size // 2
            diagonal = True
        row += rhombus_density
    
    # Color Assimation Phase   
    mask = (opacity * mask).astype(np.float32)
    inverse_mask = 1 - mask

    I = ((mask * rgb_images[idx]) + (I * inverse_mask)).astype(np.uint8)
    cv.imwrite(f"./outputs/{images_paths[idx].split(".")[0]}_colorized_rhombus.png", I)
    
    
    # Showing the result
    final_image = np.concatenate([I_clone, I], axis=1)
    final_image_resized = cv.resize(final_image, (w, h//2))
    cv.imshow("pic", final_image_resized)
    cv.waitKey(0)
    cv.destroyAllWindows()

Proccessing AmirKabir.jpg: shape = (2000, 1500)
Proccessing AUT-CE.jpg: shape = (600, 1394)


### Color Propagation

In [11]:
def gray_pixel_available(mask):
    new_mask = mask.copy()
    
    new_mask = (new_mask/np.max(new_mask)).astype(np.int8)
    
    return np.sum(new_mask) != (new_mask.shape[0]*new_mask.shape[1])

In [12]:
def get_neighbors(image, position, check_mask=False, mask=None):
    neighbors = []
    rows, cols, _ = image.shape
    row, col = position

    # Define the relative positions of the neighbors
    neighbor_positions = [(-1, 0), (1, 0), (0, -1), (0, 1), 
                          (-1, -1), (-1, 1), (1, -1), (1, 1)]

    # Iterate over each possible neighbor position
    for dr, dc in neighbor_positions:
        r, c = row + dr, col + dc
        # Check if the neighbor is within the image boundaries
        if 0 <= r < rows and 0 <= c < cols:
            if check_mask:
                if mask[r,c] == 1:
                    neighbors.append(image[r, c])
            else:
                neighbors.append(image[r, c])

    return neighbors

In [ ]:
fully_colorized_images = []

for idx, colorized_image in enumerate(colorized_images):
    image = colorized_image.copy()
    image_clone = image.copy()
    mask = masks[idx][:, :, 0]
    gray_area_mask = 1 - mask

    h, w = mask.shape

    Round = 1
    print(f"Proccessing {images_paths[idx]}\n")

    while gray_pixel_available(mask):

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))

        new_mask = cv.morphologyEx(mask, cv.MORPH_DILATE, kernel)
        
        diff = new_mask - mask
        for i in range(h):
            for j in range(w):
                if diff[i, j] == 1:
                    neighbors = get_neighbors(
                        image=image, position=(i, j), check_mask=True, mask=mask
                    )
                    
                    size = len(neighbors)

                    avg_b = int(sum(el[0] for el in neighbors) / size)
                    avg_g = int(sum(el[1] for el in neighbors) / size)
                    avg_r = int(sum(el[2] for el in neighbors) / size)

                    values = np.array([avg_b, avg_g, avg_r])
                    image[i, j, :] = values
            print(f"Round {Round} Progress: {(i+1)*100/h:.2f}%    \r", end="")
        
        print()
        Round += 1
                  

        I = np.concatenate([image, image_clone], axis=1)
        I = cv.resize(I, (w, h // 2))

        cv.imshow("pic", I)
        cv.waitKey(1)

        mask = new_mask

        print(f"[ Overall Progress: {(np.sum(mask)/(mask.shape[0]*mask.shape[1]))*100:.2f}% ]")

    print(f"\nColorization on {images_paths[idx]} Finished.\n")
    
    fully_colorized_images.append(image)
    cv.imwrite(f"./outputs/{images_paths[idx].split(".")[0]}_fullycolorized.png", image)

    cv.waitKey(0)
    cv.destroyAllWindows()

Proccessing AmirKabir.jpg

Round 1 Progress: 100.00%    
[ Overall Progress: 85.65% ]
Round 2 Progress: 100.00%    
[ Overall Progress: 99.73% ]
Round 3 Progress: 100.00%    
[ Overall Progress: 99.89% ]
Round 4 Progress: 100.00%    
[ Overall Progress: 99.96% ]
Round 5 Progress: 100.00%    
[ Overall Progress: 99.99% ]
Round 6 Progress: 100.00%    
[ Overall Progress: 100.00% ]
Round 7 Progress: 100.00%    
[ Overall Progress: 100.00% ]

Colorization on AmirKabir.jpg Finished.

Proccessing AUT-CE.jpg

Round 1 Progress: 100.00%    
[ Overall Progress: 85.54% ]
Round 2 Progress: 100.00%    
[ Overall Progress: 99.68% ]
Round 3 Progress: 100.00%    
[ Overall Progress: 99.90% ]
Round 4 Progress: 100.00%    
[ Overall Progress: 99.95% ]
Round 5 Progress: 100.00%    
[ Overall Progress: 99.99% ]
Round 6 Progress: 100.00%    
[ Overall Progress: 100.00% ]
Round 7 Progress: 100.00%    
[ Overall Progress: 100.00% ]

Colorization on AUT-CE.jpg Finished.



In [17]:
for idx in range(len(images_paths)):
    print(f"Comparing Original with Colorized Image for {images_paths[idx]}:")
    print(f"\tSSIM score: {ssim(im1=rgb_images[idx], im2=fully_colorized_images[idx], channel_axis=2)}")
    print(f"\tPSNR score: {psnr(image_true=rgb_images[idx], image_test=fully_colorized_images[idx])}")
    print()

Comparing Original with Colorized Image for AmirKabir.jpg:
	SSIM score: 0.9209619069271836
	PSNR score: 34.264534800986425

Comparing Original with Colorized Image for AUT-CE.jpg:
	SSIM score: 0.7778338726895614
	PSNR score: 22.023846005775162

